JavaScript callbacks

While the main goal of Bokeh is to provide a path to create rich interactive visualizations in the browser purely from Python, there will always be specialized use-cases that are outside the capabilities of the core library. For this reason, Bokeh provides different ways for users to supply custom JavaScript when necessary, so that users may add custom or specialized behaviors in response to property changes and other events.

One mechanism is the ability to add entire new custom extension models, as described in Extending Bokeh. However, it is also possible to supply small snippets of JavaScript as callbacks to use, e.g when property values change or when UI or other events occur. This kind of callback can be used to add interesting interactions to Bokeh documents without requiring a Bokeh server (but can also be used in conjunction with a Bokeh server).

property callbacks is responding to changes to widgets. The code below shows an example of CustomJS set on a slider Widget that changes the source of a plot when the slider is used.

These CustomJS callbacks can be attached to property change events on any Bokeh model, using the js_on_change method of Bokeh models

In [2]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import Figure,  output_notebook, show


# Enable the Bokeh output in the Jupyter notebook
output_notebook()


x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = Figure(width=400, height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const f = cb_obj.value
    const x = data['x']
    const y = data['y']
    for (let i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

Loading BokehJS ...

It should be mentioned that the first parameter to js_on_change is actually the name of a BokehJS event. The full format for a property change event is, for example, "change:start", but Bokeh will automatically convert any property name into one of these BokehJS change events for you. Additionally, some Bokeh models have additional specialized events. For example, the ColumnDataSource also supports "patch" and "stream" events, for executing CustomJS callbacks whenever the data source is patched or streamed to.

Below is an example that shows how to attach a CustomJS callback to a Slider widget, so that whenever the slider value updates, the callback is executed to update some data:

p = figure()

#execute a callback whenever p.x_range.start changes

p.x_range.js_on_change('start', callback)

CustomJS callbacks
To supply a snippet of JavaScript code that should be executed (in the browser) when some event occurs, use the CustomJS model:

from bokeh.models.callbacks import CustomJS

callback = CustomJS(args=dict(xr=plot.x_range), code="""

// JavaScript code goes here

const a = 10;

// the model that triggered the callback is cb_obj:
const b = cb_obj.value;

// models passed as args are automagically available
xr.start = a;
xr.end = b;

""")

Note that in addition to the code property, CustomJS also accepts an args property that maps string names to Bokeh models. Any Bokeh models that are configured in args (on the “Python side”) will automatically be available to the JavaScript code by the corresponding name. Additionally, the model that triggers the callback (that is the model that the callback is attached to) will be available as cb_obj.